<a href="https://colab.research.google.com/github/thestbobo/DLML_FL_project/blob/main/colab_runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Environment Setup**

In [ ]:
# Clone GitHub repository
!git clone https://github.com/thestbobo/DLML_FL_project.git
%cd DLML_FL_project

# Install dependencies
!pip install -r requirements.txt

**Environment check**

In [ ]:
import torch
print("CUDA avaiable:", torch.cuda.is_available())

**Load Configuration (hyperparameters) and Paths (in/out)**

-Example file for config.yaml

batch_size: 64

lr: 0.01

-Example of code usage:

print(config["batch_size"]) --> 64


In [ ]:
import yaml
from pathlib import Path

# load YAML config
with open("experimentS/config.yaml") as f:
  config = yaml.safe_load(f)

# Setup paths and device
DATA_DIR = Path("./data")
DEVICE= torch.device("cuda" if torch.cuda.is_available() else "cpu")

**Data preparation**

In [ ]:
from data.prepare_data import get_cifar100_loaders

train_loader, val_loader, test_loader = get_cifar100_loaders(data_dir=DATA_DIR, val_split=config["val_split"], config["batch_size"], config["num_workers"])
for batch in train_loader:
    x, y = batch
    print("Train batch shape:", x.shape, y.shape)
    break
